### Word Group In Youtube Sentence

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
adjust_text_word_ratio = 5
shift = 0.3  # sentence time shift

# func select
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number
sentence_word_count_number = 5   # False for selecting word group according to max word group length

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/2-Word Group In Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [6]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [7]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [8]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988210,karneleme,5
988211,karnaya,5
988212,dörtlümüzün,5
988213,karnavalınız,5


In [9]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [10]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
5,mi,5362714
6,o,5013838
7,ben,4908913
8,de,4880315
9,çok,4852169


In [11]:
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,english_word
0,abaküs,abacus
1,abluka,blockade
2,absorbe,absorb
3,absürt,absurd
4,açelya,azalea
...,...,...
1773,zebra,zebra
1774,zikzak,zigzag
1775,zombi,zombie
1776,zooloji,zoology


In [12]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,english_word


In [13]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [14]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [15]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))
len(word_list)

28

#### Youtube Sentence Analysis

In [16]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [17]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [18]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
3063204,3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [19]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [20]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"büyük, bunlar, araçlar, normalde, kamyon, özgü...","için, daha"
1,1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"arka, bağlamak, tarafı, olmayacak, kolay, burcu",
2,2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"yarım, özgür, attıracağım, tur, arabaya",
3,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"tamam, et, biraz, oldu, devam, burcu",daha
4,4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"kalınlığı, tekere, kar, şimdilik, iki, artarsa...",ama
...,...,...,...,...,...,...,...,...
3063204,3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA,9.2,"sağ, açıklamalar, oynatma, önerilen, üst, vide...",ve
3063205,3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA,9.2,"deseni, bunlardan, kenar, kısmına, dilediğiniz...",bu
3063206,3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA,6.8,"eğitim, ibaretti, sonraki, videoda, lütfen, el...",bir
3063207,3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA,9.2,"iyi, kendinize, unutmayın, bakın, abone, kanal...",çok


In [21]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"büyük, bunlar, araçlar, normalde, kamyon, özgü...","için, daha"
1,1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"arka, bağlamak, tarafı, olmayacak, kolay, burcu",NaN
2,2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"yarım, özgür, attıracağım, tur, arabaya",NaN
3,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"tamam, et, biraz, oldu, devam, burcu",daha
4,4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"kalınlığı, tekere, kar, şimdilik, iki, artarsa...",ama
...,...,...,...,...,...,...,...,...
3063204,3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA,9.2,"sağ, açıklamalar, oynatma, önerilen, üst, vide...",ve
3063205,3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA,9.2,"deseni, bunlardan, kenar, kısmına, dilediğiniz...",bu
3063206,3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA,6.8,"eğitim, ibaretti, sonraki, videoda, lütfen, el...",bir
3063207,3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA,9.2,"iyi, kendinize, unutmayın, bakın, abone, kanal...",çok


In [22]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,48,713.693,714.739,özgür değil,8V9tq1pe8eI,50.1,özgür,değil
1,51,726.015,726.783,köpek değil bu,8V9tq1pe8eI,66.8,köpek,"bu, değil"
2,83,833.164,834.932,daha çok aşağıda,8V9tq1pe8eI,66.8,aşağıda,"çok, daha"
3,94,877.823,880.649,evet ve enerjiyi aldı,8V9tq1pe8eI,50.1,"enerjiyi, aldı","ve, evet"
4,104,922.963,924.927,çok soruyorlar bu kadar,8V9tq1pe8eI,75.1,soruyorlar,"çok, kadar, bu"
...,...,...,...,...,...,...,...,...
241909,3062869,478.137,479.795,ve bir sonraki,3SuLoQRbZc8,66.8,sonraki,"ve, bir"
241910,3062873,486.678,487.711,ve modelin,3SuLoQRbZc8,50.1,modelin,ve
241911,3062887,541.147,541.647,ve,3SuLoQRbZc8,100.1,NaN,ve
241912,3063084,297.065,299.935,ve bu aslında,7HrxKm4FDw8,66.8,aslında,"ve, bu"


In [23]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
22,441,48.700,51.700,ve böyle çok acayip bir şey daha var bu cheese...,cQjX9DzXTw0,58.4,"acayip, ile, cheesecake, ilgili, böyle","ve, bir, var, şey, çok, daha, bu",bir şey daha var bu
75,1745,121.801,124.395,bu kadar evet bu kadar,dSG4sD2bmXg,60.1,NaN,"kadar, bu, evet",bu kadar evet bu kadar
78,1802,479.629,483.654,berlin yaraladı mı seni bana bir şey yapamadı ...,dSG4sD2bmXg,50.1,"iz, bende, yaraladı, yapamadı, bıraktı, berlin","seni, bir, mı, bana, şey, evet",mı seni bana bir şey
133,2673,933.887,936.191,yani yer değerlendirme yok bir şey yok bu,q8_P_1xvt4w,50.1,"yer, değerlendirme, yani","yok, bu, şey, bir",yok bir şey yok bu
508,7008,966.655,969.681,tamam cesaretliyiz ama bu kadar da değil yani,kSlAhrlx1lw,62.6,"tamam, cesaretliyiz, yani","kadar, ama, değil, bu, da",ama bu kadar da değil
...,...,...,...,...,...,...,...,...,...
240771,3035361,5092.440,5093.720,bir şey yok değil mi,qvPu7qxSnes,100.1,NaN,"mi, bir, şey, değil, yok",bir şey yok değil mi
240913,3036940,13.660,16.660,bir şey ama daha çok arıza vs yaparsa o zaman ...,LtxWC78DEaw,54.6,"insanlar, yaparsa, vs, arıza, zaman","bir, şey, ama, çok, daha, o",bir şey ama daha çok
241099,3040214,281.612,285.903,ve gördüğünüz gibi harika görseli ve bir o kad...,zcvS4jzdA4M,50.1,"gördüğünüz, oldu, lezzetli, görseli, harika","ve, bir, kadar, gibi, o, da",ve bir o kadar da
241272,3044838,49.292,53.000,mevzu o değil de bir de insanlar ortalama,Aq7s33k_17A,50.1,"ortalama, mevzu, insanlar","de, o, değil, bir",o değil de bir de


In [24]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,bir şey daha var bu,49.634426,50.667213,ve böyle çok acayip bir şey daha var bu cheese...,cQjX9DzXTw0
1,bu kadar evet bu kadar,121.801000,124.395000,bu kadar evet bu kadar,dSG4sD2bmXg
2,mı seni bana bir şey,480.530119,481.851761,berlin yaraladı mı seni bana bir şey yapamadı ...,dSG4sD2bmXg
3,yok bir şey yok bu,935.123293,936.191000,yani yer değerlendirme yok bir şey yok bu,q8_P_1xvt4w
4,ama bu kadar da değil,967.865400,969.412022,tamam cesaretliyiz ama bu kadar da değil yani,kSlAhrlx1lw
...,...,...,...,...,...
830,bir şey yok değil mi,5092.440000,5093.720000,bir şey yok değil mi,qvPu7qxSnes
831,bir şey ama daha çok,13.660000,14.826667,bir şey ama daha çok arıza vs yaparsa o zaman ...,LtxWC78DEaw
832,ve bir o kadar da,283.790508,285.044800,ve gördüğünüz gibi harika görseli ve bir o kad...,zcvS4jzdA4M
833,o değil de bir de,49.744195,51.462537,mevzu o değil de bir de insanlar ortalama,Aq7s33k_17A


In [25]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,bir şey daha var bu,49.334426,50.967213,ve böyle çok acayip bir şey daha var bu cheese...,cQjX9DzXTw0
1,bu kadar evet bu kadar,121.501000,124.695000,bu kadar evet bu kadar,dSG4sD2bmXg
2,mı seni bana bir şey,480.230119,482.151761,berlin yaraladı mı seni bana bir şey yapamadı ...,dSG4sD2bmXg
3,yok bir şey yok bu,934.823293,936.491000,yani yer değerlendirme yok bir şey yok bu,q8_P_1xvt4w
4,ama bu kadar da değil,967.565400,969.712022,tamam cesaretliyiz ama bu kadar da değil yani,kSlAhrlx1lw
...,...,...,...,...,...
830,bir şey yok değil mi,5092.140000,5094.020000,bir şey yok değil mi,qvPu7qxSnes
831,bir şey ama daha çok,13.360000,15.126667,bir şey ama daha çok arıza vs yaparsa o zaman ...,LtxWC78DEaw
832,ve bir o kadar da,283.490508,285.344800,ve gördüğünüz gibi harika görseli ve bir o kad...,zcvS4jzdA4M
833,o değil de bir de,49.444195,51.762537,mevzu o değil de bir de insanlar ortalama,Aq7s33k_17A


In [26]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,bir şey daha var bu,49,51,ve böyle çok acayip bir şey daha var bu cheese...,cQjX9DzXTw0
1,bu kadar evet bu kadar,122,125,bu kadar evet bu kadar,dSG4sD2bmXg
2,mı seni bana bir şey,480,482,berlin yaraladı mı seni bana bir şey yapamadı ...,dSG4sD2bmXg
3,yok bir şey yok bu,935,936,yani yer değerlendirme yok bir şey yok bu,q8_P_1xvt4w
4,ama bu kadar da değil,968,970,tamam cesaretliyiz ama bu kadar da değil yani,kSlAhrlx1lw
...,...,...,...,...,...
830,bir şey yok değil mi,5092,5094,bir şey yok değil mi,qvPu7qxSnes
831,bir şey ama daha çok,13,15,bir şey ama daha çok arıza vs yaparsa o zaman ...,LtxWC78DEaw
832,ve bir o kadar da,283,285,ve gördüğünüz gibi harika görseli ve bir o kad...,zcvS4jzdA4M
833,o değil de bir de,49,52,mevzu o değil de bir de insanlar ortalama,Aq7s33k_17A


In [27]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,hayır hayır hayır değil evet değil ama,4818,4822,hayır hayır hayır değil evet değil ama,VjS5hmgV4qI
1,yok değil mi bir şey yok değil mi,3807,3809,yok değil mi bir şey yok değil mi,XWpMMIj8REs
2,için mi bu kadar ben ben ben ben,6957,6960,bir tır için mi bu kadar ben ben ben ben yani ...,vkuPBblXoBk
3,yok çok değil o kadar da değil,4267,4268,yok çok değil o kadar da değil,ZSX8Onpr7Qw
4,yok çok değil o kadar da değil,4267,4268,yok çok değil o kadar da değil,IGJVw1HjIHc
...,...,...,...,...,...
830,ne o ne de ben,5182,5183,ne o ne de ben hiç kimseye hiç bir şey etmedik,w4CQ0fKNBeU
831,ne var o da mı,815,816,ne var o da mı şüpheli,D9BypvVyS4Y
832,ne o sen de mi,4640,4642,ne o sen de mi seviyorsun uçakları,Fuv6bYBPMb0
833,ne o ne de sen,7048,7050,ne o ne de sen benim umrumda değilsiniz,dPMiNPHlbLU


In [28]:
df_word_group_time_loc_in_sent_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_sent_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_sent_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,hayır hayır hayır değil evet değil ama,4818,4822,hayır hayır hayır değil evet değil ama,VjS5hmgV4qI,https://www.youtube.com/watch?v=VjS5hmgV4qI&t=...
1,yok değil mi bir şey yok değil mi,3807,3809,yok değil mi bir şey yok değil mi,XWpMMIj8REs,https://www.youtube.com/watch?v=XWpMMIj8REs&t=...
2,için mi bu kadar ben ben ben ben,6957,6960,bir tır için mi bu kadar ben ben ben ben yani ...,vkuPBblXoBk,https://www.youtube.com/watch?v=vkuPBblXoBk&t=...
3,yok çok değil o kadar da değil,4267,4268,yok çok değil o kadar da değil,ZSX8Onpr7Qw,https://www.youtube.com/watch?v=ZSX8Onpr7Qw&t=...
4,yok çok değil o kadar da değil,4267,4268,yok çok değil o kadar da değil,IGJVw1HjIHc,https://www.youtube.com/watch?v=IGJVw1HjIHc&t=...
...,...,...,...,...,...,...
830,ne o ne de ben,5182,5183,ne o ne de ben hiç kimseye hiç bir şey etmedik,w4CQ0fKNBeU,https://www.youtube.com/watch?v=w4CQ0fKNBeU&t=...
831,ne var o da mı,815,816,ne var o da mı şüpheli,D9BypvVyS4Y,https://www.youtube.com/watch?v=D9BypvVyS4Y&t=...
832,ne o sen de mi,4640,4642,ne o sen de mi seviyorsun uçakları,Fuv6bYBPMb0,https://www.youtube.com/watch?v=Fuv6bYBPMb0&t=...
833,ne o ne de sen,7048,7050,ne o ne de sen benim umrumda değilsiniz,dPMiNPHlbLU,https://www.youtube.com/watch?v=dPMiNPHlbLU&t=...


In [29]:
df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)

In [30]:
df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Youtube Videoid Text Analysis

In [31]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [32]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [33]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [34]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

,index,video_id,sentence
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...
...,...,...,...
18361,18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...
18362,18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...
18363,18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...
18364,18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...


In [35]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

,video_id,start_time
0,007uL_OBtPE,6.680
1,00Dl6tNNlZ8,36.803
2,00QJHZSg0UE,0.440
3,00b3ov_NFSI,0.000
4,00fr8yyBQYo,18.000
...,...,...
18361,zzOL9mRRU0c,9.333
18362,zzWQXUKr9OA,1.896
18363,zzXTIIo7EvM,36.932
18364,zzZfnY7Zjf0,0.000


In [36]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

,video_id,end_time
0,007uL_OBtPE,136.640
1,00Dl6tNNlZ8,813.903
2,00QJHZSg0UE,27.460
3,00b3ov_NFSI,42.400
4,00fr8yyBQYo,413.000
...,...,...
18361,zzOL9mRRU0c,533.254
18362,zzWQXUKr9OA,27.969
18363,zzXTIIo7EvM,1591.680
18364,zzZfnY7Zjf0,15.000


In [37]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

,video_id,start_time,end_time
0,007uL_OBtPE,6.680,136.640
1,00Dl6tNNlZ8,36.803,813.903
2,00QJHZSg0UE,0.440,27.460
3,00b3ov_NFSI,0.000,42.400
4,00fr8yyBQYo,18.000,413.000
...,...,...,...
18361,zzOL9mRRU0c,9.333,533.254
18362,zzWQXUKr9OA,1.896,27.969
18363,zzXTIIo7EvM,36.932,1591.680
18364,zzZfnY7Zjf0,0.000,15.000


In [38]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

,index,video_id,sentence,start_time,end_time
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,6.680,136.640
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,36.803,813.903
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,0.440,27.460
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,0.000,42.400
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...,18.000,413.000
...,...,...,...,...,...
18361,18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,9.333,533.254
18362,18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,1.896,27.969
18363,18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,36.932,1591.680
18364,18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,0.000,15.000


In [39]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [40]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,7.2,"mısır, eridikten, siz, şöyle, karıştırıyoruz, ...","de, ve, var, için, kadar, bunu, ama, gibi, çok..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,1.6,"çıkarmasınlar, getirmek, bize, kaldırmalıyız, ...","de, şey, onu, için, ne, değil, evet, da, mi, v..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,11.2,"yakışıyorsunuz, gereğini, h, idünyalara, biraz...","de, mi, ben, sen, ne"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,15.4,"usta, abla, barışırsınız, boşalır, yabancı, de...","mi, onu, bir, var, kadar, gibi, ama, sen, ne, ..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.1,"entertaining, place, infected, noticed, commen...",
...,...,...,...,...,...
18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,2.4,"zenginim, kivi, rahatlıkla, bize, bakımından, ...","de, seni, ve, bir, var, için, bana, ben, ama, ..."
18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,16.5,"bırak, kızınız, varsa, bilmem, nermin, ya, baş...","de, mi, seni, bana, ben, gibi, bu, o, da"
18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,1.1,"villamızı, girmesi, sevdiğim, koridorun, bunun...","de, şey, onu, için, bana, ne, evet, da, mi, ve..."
18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,14.1,"vardı, ihtiyacım, yaptığıma, edebileceğim, tas...","ve, bir, için, kadar, ben, çok"


In [41]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,7.2,"mısır, eridikten, siz, şöyle, karıştırıyoruz, ...","de, ve, var, için, kadar, bunu, ama, gibi, çok..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,1.6,"çıkarmasınlar, getirmek, bize, kaldırmalıyız, ...","de, şey, onu, için, ne, değil, evet, da, mi, v..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,11.2,"yakışıyorsunuz, gereğini, h, idünyalara, biraz...","de, mi, ben, sen, ne"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,15.4,"usta, abla, barışırsınız, boşalır, yabancı, de...","mi, onu, bir, var, kadar, gibi, ama, sen, ne, ..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.1,"entertaining, place, infected, noticed, commen...",NaN
...,...,...,...,...,...
18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,2.4,"zenginim, kivi, rahatlıkla, bize, bakımından, ...","de, seni, ve, bir, var, için, bana, ben, ama, ..."
18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,16.5,"bırak, kızınız, varsa, bilmem, nermin, ya, baş...","de, mi, seni, bana, ben, gibi, bu, o, da"
18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,1.1,"villamızı, girmesi, sevdiğim, koridorun, bunun...","de, şey, onu, için, bana, ne, evet, da, mi, ve..."
18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,14.1,"vardı, ihtiyacım, yaptığıma, edebileceğim, tas...","ve, bir, için, kadar, ben, çok"


In [42]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,7.2,"mısır, eridikten, siz, şöyle, karıştırıyoruz, ...","de, ve, var, için, kadar, bunu, ama, gibi, çok..."
1,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,11.2,"yakışıyorsunuz, gereğini, h, idünyalara, biraz...","de, mi, ben, sen, ne"
2,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,15.4,"usta, abla, barışırsınız, boşalır, yabancı, de...","mi, onu, bir, var, kadar, gibi, ama, sen, ne, ..."
3,00pHXwzua50,i hlas suresi meali rahman ve rahim olan allah...,10.1,"adıyla, rahman, suresi, rahim, hlas, meali, i,...",ve
4,00rzeUrFM6E,herkese merhaba feyzadan lezzetler aldım buray...,5.1,"sağ, yorumlarınızı, rahatlıkla, bölümüne, aras...","de, mi, ve, bir, için, kadar, şey, ben, ama, n..."
...,...,...,...,...,...
6958,zxXQRn3OLdc,hoş geldin nasılsın beni sorarsan elhamdülilla...,6.6,"sevdiğim, omlet, fırçam, gerçekten, arasına, s...","de, seni, ve, onu, bir, var, için, gibi, şey, ..."
6959,zzBj92_1ITE,denebilir ki habercinin işi hayatın peşine tak...,7.2,"kurşunlarına, meslektaşım, terör, işi, az, ya,...","gibi, ama, bir"
6960,zzNekJQxa3s,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,15.1,"atıp, vlog, eden, yılbaşı, çekimimde, merhaba,...","ve, için, ben, çok, daha, bu"
6961,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,16.5,"bırak, kızınız, varsa, bilmem, nermin, ya, baş...","de, mi, seni, bana, ben, gibi, bu, o, da"


In [43]:
if word_group_adjust_select:
    df_adjust_text_ratio = df_exract_word_group(df_adjust_text_ratio, "sentence", "video_id", word_list, sent_len=word_group_adjust_select, sent_len_num=sentence_word_count_number)
else:
    df_adjust_text_ratio.loc[:,"search_string"] = df_adjust_text_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
    df_adjust_text_ratio = df_adjust_text_ratio[~(df_adjust_text_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
    
df_adjust_text_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,video_id,sentence,word_ratio,different_word,intersect_word,search_string
54,0WRd96qvAAg,tadımlık bilgiler kanalından herkese merhaba b...,5.5,"figürü, tadımlık, kadınlar, sadece, çocuk, res...","de, ve, bir, için, kadar, ama, gibi, çok, bu, ...",ama bir o kadar da
77,0hk9BhGgIEI,of bayağı yoruldum zahra bir çay koy da içelim...,6.9,"aklı, kesemem, normalde, nehir, olması, içelim...","de, şey, onu, bana, ne, da, seni, var, bunu, b...",bir şey yok bir şey yok
116,16k333qUijs,ben şimdi ne diyeceğim kızlara o kadar hava ya...,8.2,"gideceksin, geliyorsa, olmuş, büdü, konuşalım,...","de, şey, onu, için, bana, ne, da, mi, seni, ve...",mı ben de seni bu
132,1BhpEtCO4u4,selamın aleyküm gerdanı gördünüz fakat yapamad...,5.2,"fırsat, tuzu, saralım, aleyküm, sadece, altı, ...","ve, bir, kadar, bunu, gibi, çok, daha, bu, o, da",ve bir o kadar da
324,31GCcyoN4zE,ne zaman ne zaman çok yazmaya başlasam saçmalı...,13.7,"ya, ölümüm, yazmaya, geçiyorum, git, hatırlasa...","de, ve, seni, bir, sen, ne, çok, daha, yok",bir daha bir daha ve bir daha seni
...,...,...,...,...,...,...
6738,xyILCkW5R6E,kardeşim lan kalk i yi ki bir hesabı ödedik ab...,7.4,"dertlisin, nereden, gücünüz, efendim, kırıldım...","de, şey, bana, ne, değil, da, mi, seni, ve, va...",var mı o da var
6844,yrqg6fdAnV8,ah nehir nerede kaldın be kızım ya of gelmedi ...,9.5,"meydanda, nehir, nşallah, bilir, hiç, nerede, ...","de, mi, bir, var, kadar, gibi, şey, ben, ne, d...",bir şey yok gibi yok ben
6847,ysSMerd7cOs,merhabalar ben yusuf aktaş bir çoğunuz reynmen...,5.7,"süre, reyynmmeeeen, gerçekten, çocuk, vaz, bur...","de, şey, için, bana, ne, değil, da, mi, ve, va...",bir şey var ben bunu
6896,zKdjR4mapsc,niye h l gelmediler ah güzelim gelirler anneci...,10.0,"açmıştır, göstereyim, babacığım, musun, omzuna...","de, mi, ve, bir, ben, ama, sen, değil, ne, çok...",mi evet sen de çok


In [44]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

,search_string,video_id,sentence
54,ama bir o kadar da,0WRd96qvAAg,tadımlık bilgiler kanalından herkese merhaba b...
77,bir şey yok bir şey yok,0hk9BhGgIEI,of bayağı yoruldum zahra bir çay koy da içelim...
116,mı ben de seni bu,16k333qUijs,ben şimdi ne diyeceğim kızlara o kadar hava ya...
132,ve bir o kadar da,1BhpEtCO4u4,selamın aleyküm gerdanı gördünüz fakat yapamad...
324,bir daha bir daha ve bir daha seni,31GCcyoN4zE,ne zaman ne zaman çok yazmaya başlasam saçmalı...
...,...,...,...
6738,var mı o da var,xyILCkW5R6E,kardeşim lan kalk i yi ki bir hesabı ödedik ab...
6844,bir şey yok gibi yok ben,yrqg6fdAnV8,ah nehir nerede kaldın be kızım ya of gelmedi ...
6847,bir şey var ben bunu,ysSMerd7cOs,merhabalar ben yusuf aktaş bir çoğunuz reynmen...
6896,mi evet sen de çok,zKdjR4mapsc,niye h l gelmediler ah güzelim gelirler anneci...


In [45]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [46]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

,search_string,video_id,sentence,start_time,end_time
0,ama bir o kadar da,0WRd96qvAAg,tadımlık bilgiler kanalından herkese merhaba b...,10.540,301.980
1,bir şey yok bir şey yok,0hk9BhGgIEI,of bayağı yoruldum zahra bir çay koy da içelim...,2.855,155.581
2,mı ben de seni bu,16k333qUijs,ben şimdi ne diyeceğim kızlara o kadar hava ya...,3.400,184.463
3,ve bir o kadar da,1BhpEtCO4u4,selamın aleyküm gerdanı gördünüz fakat yapamad...,0.020,401.040
4,bir daha bir daha ve bir daha seni,31GCcyoN4zE,ne zaman ne zaman çok yazmaya başlasam saçmalı...,5.000,76.160
...,...,...,...,...,...
129,var mı o da var,xyILCkW5R6E,kardeşim lan kalk i yi ki bir hesabı ödedik ab...,15.520,163.640
130,bir şey yok gibi yok ben,yrqg6fdAnV8,ah nehir nerede kaldın be kızım ya of gelmedi ...,2.204,137.869
131,bir şey var ben bunu,ysSMerd7cOs,merhabalar ben yusuf aktaş bir çoğunuz reynmen...,0.000,190.900
132,mi evet sen de çok,zKdjR4mapsc,niye h l gelmediler ah güzelim gelirler anneci...,2.584,78.324


In [47]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,ama bir o kadar da,32.857477,36.608314,tadımlık bilgiler kanalından herkese merhaba b...,0WRd96qvAAg
1,bir şey yok bir şey yok,95.275214,97.556068,of bayağı yoruldum zahra bir çay koy da içelim...,0hk9BhGgIEI
2,mı ben de seni bu,33.322722,35.256503,ben şimdi ne diyeceğim kızlara o kadar hava ya...,16k333qUijs
3,ve bir o kadar da,86.361140,92.077104,selamın aleyküm gerdanı gördünüz fakat yapamad...,1BhpEtCO4u4
4,bir daha bir daha ve bir daha seni,40.298364,47.057625,ne zaman ne zaman çok yazmaya başlasam saçmalı...,31GCcyoN4zE
...,...,...,...,...,...
129,var mı o da var,31.281152,32.761481,kardeşim lan kalk i yi ki bir hesabı ödedik ab...,xyILCkW5R6E
130,bir şey yok gibi yok ben,84.282944,88.492121,ah nehir nerede kaldın be kızım ya of gelmedi ...,yrqg6fdAnV8
131,bir şey var ben bunu,56.730405,58.334607,merhabalar ben yusuf aktaş bir çoğunuz reynmen...,ysSMerd7cOs
132,mi evet sen de çok,73.578843,75.403904,niye h l gelmediler ah güzelim gelirler anneci...,zKdjR4mapsc


In [48]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,ama bir o kadar da,32.557477,36.908314,tadımlık bilgiler kanalından herkese merhaba b...,0WRd96qvAAg
1,bir şey yok bir şey yok,94.975214,97.856068,of bayağı yoruldum zahra bir çay koy da içelim...,0hk9BhGgIEI
2,mı ben de seni bu,33.022722,35.556503,ben şimdi ne diyeceğim kızlara o kadar hava ya...,16k333qUijs
3,ve bir o kadar da,86.061140,92.377104,selamın aleyküm gerdanı gördünüz fakat yapamad...,1BhpEtCO4u4
4,bir daha bir daha ve bir daha seni,39.998364,47.357625,ne zaman ne zaman çok yazmaya başlasam saçmalı...,31GCcyoN4zE
...,...,...,...,...,...
129,var mı o da var,30.981152,33.061481,kardeşim lan kalk i yi ki bir hesabı ödedik ab...,xyILCkW5R6E
130,bir şey yok gibi yok ben,83.982944,88.792121,ah nehir nerede kaldın be kızım ya of gelmedi ...,yrqg6fdAnV8
131,bir şey var ben bunu,56.430405,58.634607,merhabalar ben yusuf aktaş bir çoğunuz reynmen...,ysSMerd7cOs
132,mi evet sen de çok,73.278843,75.703904,niye h l gelmediler ah güzelim gelirler anneci...,zKdjR4mapsc


In [49]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,ama bir o kadar da,33,37,tadımlık bilgiler kanalından herkese merhaba b...,0WRd96qvAAg
1,bir şey yok bir şey yok,95,98,of bayağı yoruldum zahra bir çay koy da içelim...,0hk9BhGgIEI
2,mı ben de seni bu,33,36,ben şimdi ne diyeceğim kızlara o kadar hava ya...,16k333qUijs
3,ve bir o kadar da,86,92,selamın aleyküm gerdanı gördünüz fakat yapamad...,1BhpEtCO4u4
4,bir daha bir daha ve bir daha seni,40,47,ne zaman ne zaman çok yazmaya başlasam saçmalı...,31GCcyoN4zE
...,...,...,...,...,...
129,var mı o da var,31,33,kardeşim lan kalk i yi ki bir hesabı ödedik ab...,xyILCkW5R6E
130,bir şey yok gibi yok ben,84,89,ah nehir nerede kaldın be kızım ya of gelmedi ...,yrqg6fdAnV8
131,bir şey var ben bunu,56,59,merhabalar ben yusuf aktaş bir çoğunuz reynmen...,ysSMerd7cOs
132,mi evet sen de çok,73,76,niye h l gelmediler ah güzelim gelirler anneci...,zKdjR4mapsc


In [50]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,hayır hayır hayır hayır hayır hayır,27,39,ben nasıl diyeceğimi bilemiyorum hayır yok hay...,fh8Au3ZmwqI
1,bir daha bir daha ve bir daha seni,40,47,ne zaman ne zaman çok yazmaya başlasam saçmalı...,31GCcyoN4zE
2,bu değil bu da değil bu da değil,17,21,evet hazırlığım tamamlanmak üzere dışarı çıkıc...,Tld0_wJE2BU
3,o kadar da değil o kadar o kadar,99,104,nehir nehir bir durur musun lütfen nehir nehir...,WXIuUTnr6bc
4,ve bir çok şey için o kadar çok,103,112,adil hocam ne yaptınız yahu gözünü en yükseğe ...,EVexraDiTFc
...,...,...,...,...,...
129,bu bu ne bu ne,169,172,kesinlikle olmaz asla izin vermiyorum bitmesin...,bp71DOdcWeQ
130,var mı o o o,627,632,o lanet olsun hep aynı kural işler ben aşkımı ...,GaQjOktW5mE
131,o o o o o o,28,47,abone ol abone ol o o o o o o şu anda niye böy...,5Lbnm9ZYVlc
132,o o o o bu,399,404,evet herkese selamlar ben savaştığında bugün b...,R3LBj_c2KII


In [51]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,hayır hayır hayır hayır hayır hayır,27,39,ben nasıl diyeceğimi bilemiyorum hayır yok hay...,fh8Au3ZmwqI,https://www.youtube.com/watch?v=fh8Au3ZmwqI&t=27s
1,bir daha bir daha ve bir daha seni,40,47,ne zaman ne zaman çok yazmaya başlasam saçmalı...,31GCcyoN4zE,https://www.youtube.com/watch?v=31GCcyoN4zE&t=40s
2,bu değil bu da değil bu da değil,17,21,evet hazırlığım tamamlanmak üzere dışarı çıkıc...,Tld0_wJE2BU,https://www.youtube.com/watch?v=Tld0_wJE2BU&t=17s
3,o kadar da değil o kadar o kadar,99,104,nehir nehir bir durur musun lütfen nehir nehir...,WXIuUTnr6bc,https://www.youtube.com/watch?v=WXIuUTnr6bc&t=99s
4,ve bir çok şey için o kadar çok,103,112,adil hocam ne yaptınız yahu gözünü en yükseğe ...,EVexraDiTFc,https://www.youtube.com/watch?v=EVexraDiTFc&t=...
...,...,...,...,...,...,...
129,bu bu ne bu ne,169,172,kesinlikle olmaz asla izin vermiyorum bitmesin...,bp71DOdcWeQ,https://www.youtube.com/watch?v=bp71DOdcWeQ&t=...
130,var mı o o o,627,632,o lanet olsun hep aynı kural işler ben aşkımı ...,GaQjOktW5mE,https://www.youtube.com/watch?v=GaQjOktW5mE&t=...
131,o o o o o o,28,47,abone ol abone ol o o o o o o şu anda niye böy...,5Lbnm9ZYVlc,https://www.youtube.com/watch?v=5Lbnm9ZYVlc&t=28s
132,o o o o bu,399,404,evet herkese selamlar ben savaştığında bugün b...,R3LBj_c2KII,https://www.youtube.com/watch?v=R3LBj_c2KII&t=...


In [52]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [53]:
#df_word_group_time_loc_in_text_result.to_csv(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.csv", index=False)

In [54]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Word_Group_Max_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [55]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file

['Turkish_English_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx',
 'Turkish_English_Word_Group_Max_In_Youtube_Sentence_Text_28_Word6.xlsx']

In [56]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [57]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Youtube Videoid Text Adjust Result Analysis Multiprocess

In [65]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [66]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [67]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
adjust_text_word_ratio = 5
shift = 0.3  # sentence time shift

# func select
word_group_adjust_select = True  # True, False; True for selecting word group according to sentence_word_count_number
sentence_word_count_number = 5   # False for selecting word group according to all word group length

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

5


In [68]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [69]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988210,karneleme,5
988211,karnaya,5
988212,dörtlümüzün,5
988213,karnavalınız,5


In [70]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [71]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
150,buradaydı,16726
151,buysa,16063
152,miydiniz,15468
153,bunlarla,15042


In [72]:
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,english_word
0,abaküs,abacus
1,abluka,blockade
2,absorbe,absorb
3,absürt,absurd
4,açelya,azalea
...,...,...
1773,zebra,zebra
1774,zikzak,zigzag
1775,zombi,zombie
1776,zooloji,zoology


In [73]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,english_word


In [74]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [75]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [76]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))
len(word_list)

155

In [77]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [78]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [79]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [80]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

,index,video_id,sentence
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...
...,...,...,...
18361,18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...
18362,18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...
18363,18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...
18364,18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...


In [81]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

,video_id,start_time
0,007uL_OBtPE,6.680
1,00Dl6tNNlZ8,36.803
2,00QJHZSg0UE,0.440
3,00b3ov_NFSI,0.000
4,00fr8yyBQYo,18.000
...,...,...
18361,zzOL9mRRU0c,9.333
18362,zzWQXUKr9OA,1.896
18363,zzXTIIo7EvM,36.932
18364,zzZfnY7Zjf0,0.000


In [82]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

,video_id,end_time
0,007uL_OBtPE,136.640
1,00Dl6tNNlZ8,813.903
2,00QJHZSg0UE,27.460
3,00b3ov_NFSI,42.400
4,00fr8yyBQYo,413.000
...,...,...
18361,zzOL9mRRU0c,533.254
18362,zzWQXUKr9OA,27.969
18363,zzXTIIo7EvM,1591.680
18364,zzZfnY7Zjf0,15.000


In [83]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

,video_id,start_time,end_time
0,007uL_OBtPE,6.680,136.640
1,00Dl6tNNlZ8,36.803,813.903
2,00QJHZSg0UE,0.440,27.460
3,00b3ov_NFSI,0.000,42.400
4,00fr8yyBQYo,18.000,413.000
...,...,...,...
18361,zzOL9mRRU0c,9.333,533.254
18362,zzWQXUKr9OA,1.896,27.969
18363,zzXTIIo7EvM,36.932,1591.680
18364,zzZfnY7Zjf0,0.000,15.000


In [84]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

,index,video_id,sentence,start_time,end_time
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,6.680,136.640
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,36.803,813.903
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,0.440,27.460
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,0.000,42.400
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...,18.000,413.000
...,...,...,...,...,...
18361,18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,9.333,533.254
18362,18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,1.896,27.969
18363,18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,36.932,1591.680
18364,18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,0.000,15.000


In [85]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list3 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list3.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [86]:
df_text_ratio_result = pd.DataFrame(list(result_list3), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,8.4,"mısır, eridikten, siz, şöyle, afiyet, karıştır...","de, onun, ve, var, kadar, için, bunu, ama, bug..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,2.7,"çıkarmasınlar, getirmek, bize, kaldırmalıyız, ...","de, buna, değiller, varsa, içinde, şey, budur,..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,15.7,"yakışıyorsunuz, gereğini, h, idünyalara, birbi...","de, mi, ben, biraz, sen, ne, benim"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,19.5,"usta, abla, barışırsınız, boşalır, yabancı, na...","değilsiniz, mısın, benim, onu, değil, ne, da, ..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.1,"entertaining, infected, noticed, comment, rese...",
...,...,...,...,...,...
18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,3.7,"zenginim, kivi, rahatlıkla, bize, bakımından, ...","de, varsa, biraz, benim, için, vardır, bana, n..."
18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,23.7,"bırak, kızınız, bilmem, nermin, ya, baştan, aş...","de, mi, seni, varsa, yoksa, birisi, bana, ben,..."
18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,2.0,"villamızı, girmesi, sevdiğim, koridorun, norma...","de, buna, vardı, bununla, şey, burası, biraz, ..."
18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,18.7,"ihtiyacım, yaptığıma, edebileceğim, taslağını,...","ve, vardı, bir, için, kadar, bunun, ben, çok"


In [87]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,8.4,"mısır, eridikten, siz, şöyle, afiyet, karıştır...","de, onun, ve, var, kadar, için, bunu, ama, bug..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,2.7,"çıkarmasınlar, getirmek, bize, kaldırmalıyız, ...","de, buna, değiller, varsa, içinde, şey, budur,..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,15.7,"yakışıyorsunuz, gereğini, h, idünyalara, birbi...","de, mi, ben, biraz, sen, ne, benim"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,19.5,"usta, abla, barışırsınız, boşalır, yabancı, na...","değilsiniz, mısın, benim, onu, değil, ne, da, ..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.1,"entertaining, infected, noticed, comment, rese...",NaN
...,...,...,...,...,...
18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,3.7,"zenginim, kivi, rahatlıkla, bize, bakımından, ...","de, varsa, biraz, benim, için, vardır, bana, n..."
18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,23.7,"bırak, kızınız, bilmem, nermin, ya, baştan, aş...","de, mi, seni, varsa, yoksa, birisi, bana, ben,..."
18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,2.0,"villamızı, girmesi, sevdiğim, koridorun, norma...","de, buna, vardı, bununla, şey, burası, biraz, ..."
18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,18.7,"ihtiyacım, yaptığıma, edebileceğim, taslağını,...","ve, vardı, bir, için, kadar, bunun, ben, çok"


In [88]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,8.4,"mısır, eridikten, siz, şöyle, afiyet, karıştır...","de, onun, ve, var, kadar, için, bunu, ama, bug..."
1,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,15.7,"yakışıyorsunuz, gereğini, h, idünyalara, birbi...","de, mi, ben, biraz, sen, ne, benim"
2,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,19.5,"usta, abla, barışırsınız, boşalır, yabancı, na...","değilsiniz, mısın, benim, onu, değil, ne, da, ..."
3,00pHXwzua50,i hlas suresi meali rahman ve rahim olan allah...,10.1,"adıyla, rahman, suresi, rahim, hlas, meali, i,...",ve
4,00rzeUrFM6E,herkese merhaba feyzadan lezzetler aldım buray...,6.0,"sağ, yorumlarınızı, rahatlıkla, bölümüne, aras...","de, varsa, şey, bugün, için, buraya, ne, da, v..."
...,...,...,...,...,...
9379,zzBj92_1ITE,denebilir ki habercinin işi hayatın peşine tak...,7.2,"kurşunlarına, meslektaşım, terör, işi, az, ya,...","bir, ama, gibi"
9380,zzNekJQxa3s,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,15.1,"atıp, vlog, eden, yılbaşı, çekimimde, merhaba,...","ve, için, ben, çok, daha, bu"
9381,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,23.7,"bırak, kızınız, bilmem, nermin, ya, baştan, aş...","de, mi, seni, varsa, yoksa, birisi, bana, ben,..."
9382,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,18.7,"ihtiyacım, yaptığıma, edebileceğim, taslağını,...","ve, vardı, bir, için, kadar, bunun, ben, çok"


In [89]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list4 = manager.list()
index_list = manager.list()
all_index_list = manager.list()
var_index_list = manager.list()
text_all_list = manager.list()  
df = df_adjust_text_ratio
df.reset_index(drop=True, inplace=True)
index_num = list(range(len(df)))
sent_len = word_group_adjust_select
sent_len_num = sentence_word_count_number 

def df_exract_word_group(index_num):    
    source_text = df.loc[index_num,"sentence"]
    opt_var = df.loc[index_num,"video_id"]
    words = re.findall(r"\w+", source_text, re.UNICODE)
    
    index_list = []
    for j in range(len(words)):
        if words[j] in word_list:
            index_list.append(j)
        else:
            pass
    all_index_list = []
    var_index_list = []
    for k in range(len(index_list)):
        try:
            var1 = index_list[k] + 1  
            var2 = index_list[k+1]
        except:
            var1 = index_list[k] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[k]
            var_index_list.append(var3)
        else:
            var3 = index_list[k]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []
    text_all_list = []
    for m in all_index_list:
        text_list = [] 
        for n in m:
            word = words[n]
            text_list.append(word)
            if sent_len:
                if len(text_list) >= sent_len_num:
                    text = " ".join(text_list)
                    text_all_list.append(text)
                else:
                    pass
            else:
               text = " ".join(text_list)
               text_all_list.append(text) 
        
    for search_string in text_all_list:
        #df_var["index"] = i
        #df_var["search_string"] = search_string
        #df_var["sentence"] = source_text
        #df_var["video_id"] = opt_var
        result_list4.append([index_num,search_string,source_text,opt_var])

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(df_exract_word_group, index_num)

In [90]:
df_adjust_text_ratio = pd.DataFrame(list(result_list4), columns=["index","search_string","sentence","video_id"])
df_adjust_text_ratio.sort_values(by="index", ascending=True, inplace=True)
df_adjust_text_ratio.reset_index(drop=True, inplace=True)
df_adjust_text_ratio.drop(["index"], axis=1, inplace=True)
df_adjust_text_ratio

,search_string,sentence,video_id
0,ne var bunda bu kadar,oğlum öyle atılır mı be allahım ya rabbim bak ...,08D35tUB0Qk
1,bir şey değil neyse onu da,merhaba arkadaşlar kanalıma hoşgeldiniz evet ş...,0LGk10C8wnI
2,bir şey değil neyse onu,merhaba arkadaşlar kanalıma hoşgeldiniz evet ş...,0LGk10C8wnI
3,ne vardı bir de bir,evet hepinize merhaba arkadaşlar bugün en beğe...,0Nh2V9nyYTY
4,ve sen de benim gibi,rahip bersisa gecegündüz vaktini allaha ibadet...,0OkjJ57GYkY
...,...,...,...
1074,bir şeyler var mı yok yok,bence sen de bizimle kalmalısın pollyanna nası...,zglsIJO4xo0
1075,bir şeyler var mı yok,bence sen de bizimle kalmalısın pollyanna nası...,zglsIJO4xo0
1076,için çok çok çok ama,merhaba arkadaşlar bugün nasılsınız ben çok iy...,znRbRpIJrCU
1077,için çok çok çok ama çok,merhaba arkadaşlar bugün nasılsınız ben çok iy...,znRbRpIJrCU


In [91]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

,search_string,video_id,sentence
0,ne var bunda bu kadar,08D35tUB0Qk,oğlum öyle atılır mı be allahım ya rabbim bak ...
1,bir şey değil neyse onu da,0LGk10C8wnI,merhaba arkadaşlar kanalıma hoşgeldiniz evet ş...
2,bir şey değil neyse onu,0LGk10C8wnI,merhaba arkadaşlar kanalıma hoşgeldiniz evet ş...
3,ne vardı bir de bir,0Nh2V9nyYTY,evet hepinize merhaba arkadaşlar bugün en beğe...
4,ve sen de benim gibi,0OkjJ57GYkY,rahip bersisa gecegündüz vaktini allaha ibadet...
...,...,...,...
1074,bir şeyler var mı yok yok,zglsIJO4xo0,bence sen de bizimle kalmalısın pollyanna nası...
1075,bir şeyler var mı yok,zglsIJO4xo0,bence sen de bizimle kalmalısın pollyanna nası...
1076,için çok çok çok ama,znRbRpIJrCU,merhaba arkadaşlar bugün nasılsınız ben çok iy...
1077,için çok çok çok ama çok,znRbRpIJrCU,merhaba arkadaşlar bugün nasılsınız ben çok iy...


In [92]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [93]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

,search_string,video_id,sentence,start_time,end_time
0,ne var bunda bu kadar,08D35tUB0Qk,oğlum öyle atılır mı be allahım ya rabbim bak ...,0.000,254.528
1,bir şey değil neyse onu da,0LGk10C8wnI,merhaba arkadaşlar kanalıma hoşgeldiniz evet ş...,0.000,892.380
2,bir şey değil neyse onu,0LGk10C8wnI,merhaba arkadaşlar kanalıma hoşgeldiniz evet ş...,0.000,892.380
3,ne vardı bir de bir,0Nh2V9nyYTY,evet hepinize merhaba arkadaşlar bugün en beğe...,0.050,486.100
4,ve sen de benim gibi,0OkjJ57GYkY,rahip bersisa gecegündüz vaktini allaha ibadet...,8.135,360.790
...,...,...,...,...,...
1074,bir şeyler var mı yok yok,zglsIJO4xo0,bence sen de bizimle kalmalısın pollyanna nası...,0.210,214.511
1075,bir şeyler var mı yok,zglsIJO4xo0,bence sen de bizimle kalmalısın pollyanna nası...,0.210,214.511
1076,için çok çok çok ama,znRbRpIJrCU,merhaba arkadaşlar bugün nasılsınız ben çok iy...,0.680,19.460
1077,için çok çok çok ama çok,znRbRpIJrCU,merhaba arkadaşlar bugün nasılsınız ben çok iy...,0.680,19.460


In [94]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,ne var bunda bu kadar,107.831473,112.213317,oğlum öyle atılır mı be allahım ya rabbim bak ...,08D35tUB0Qk
1,bir şey değil neyse onu da,839.959009,844.851635,merhaba arkadaşlar kanalıma hoşgeldiniz evet ş...,0LGk10C8wnI
2,bir şey değil neyse onu,839.959009,844.327425,merhaba arkadaşlar kanalıma hoşgeldiniz evet ş...,0LGk10C8wnI
3,ne vardı bir de bir,337.341015,341.435299,evet hepinize merhaba arkadaşlar bugün en beğe...,0Nh2V9nyYTY
4,ve sen de benim gibi,156.461996,158.543114,rahip bersisa gecegündüz vaktini allaha ibadet...,0OkjJ57GYkY
...,...,...,...,...,...
1074,bir şeyler var mı yok yok,205.069980,207.810921,bence sen de bizimle kalmalısın pollyanna nası...,zglsIJO4xo0
1075,bir şeyler var mı yok,205.069980,207.404856,bence sen de bizimle kalmalısın pollyanna nası...,zglsIJO4xo0
1076,için çok çok çok ama,11.880929,13.356500,merhaba arkadaşlar bugün nasılsınız ben çok iy...,znRbRpIJrCU
1077,için çok çok çok ama çok,11.880929,13.624786,merhaba arkadaşlar bugün nasılsınız ben çok iy...,znRbRpIJrCU


In [95]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,ne var bunda bu kadar,107.531473,112.513317,oğlum öyle atılır mı be allahım ya rabbim bak ...,08D35tUB0Qk
1,bir şey değil neyse onu da,839.659009,845.151635,merhaba arkadaşlar kanalıma hoşgeldiniz evet ş...,0LGk10C8wnI
2,bir şey değil neyse onu,839.659009,844.627425,merhaba arkadaşlar kanalıma hoşgeldiniz evet ş...,0LGk10C8wnI
3,ne vardı bir de bir,337.041015,341.735299,evet hepinize merhaba arkadaşlar bugün en beğe...,0Nh2V9nyYTY
4,ve sen de benim gibi,156.161996,158.843114,rahip bersisa gecegündüz vaktini allaha ibadet...,0OkjJ57GYkY
...,...,...,...,...,...
1074,bir şeyler var mı yok yok,204.769980,208.110921,bence sen de bizimle kalmalısın pollyanna nası...,zglsIJO4xo0
1075,bir şeyler var mı yok,204.769980,207.704856,bence sen de bizimle kalmalısın pollyanna nası...,zglsIJO4xo0
1076,için çok çok çok ama,11.580929,13.656500,merhaba arkadaşlar bugün nasılsınız ben çok iy...,znRbRpIJrCU
1077,için çok çok çok ama çok,11.580929,13.924786,merhaba arkadaşlar bugün nasılsınız ben çok iy...,znRbRpIJrCU


In [96]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,ne var bunda bu kadar,108,113,oğlum öyle atılır mı be allahım ya rabbim bak ...,08D35tUB0Qk
1,bir şey değil neyse onu da,840,845,merhaba arkadaşlar kanalıma hoşgeldiniz evet ş...,0LGk10C8wnI
2,bir şey değil neyse onu,840,845,merhaba arkadaşlar kanalıma hoşgeldiniz evet ş...,0LGk10C8wnI
3,ne vardı bir de bir,337,342,evet hepinize merhaba arkadaşlar bugün en beğe...,0Nh2V9nyYTY
4,ve sen de benim gibi,156,159,rahip bersisa gecegündüz vaktini allaha ibadet...,0OkjJ57GYkY
...,...,...,...,...,...
1074,bir şeyler var mı yok yok,205,208,bence sen de bizimle kalmalısın pollyanna nası...,zglsIJO4xo0
1075,bir şeyler var mı yok,205,208,bence sen de bizimle kalmalısın pollyanna nası...,zglsIJO4xo0
1076,için çok çok çok ama,12,14,merhaba arkadaşlar bugün nasılsınız ben çok iy...,znRbRpIJrCU
1077,için çok çok çok ama çok,12,14,merhaba arkadaşlar bugün nasılsınız ben çok iy...,znRbRpIJrCU


In [97]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,misiniz içinde bir şeyler vardı ama içinde bir...,1420,1445,bugün hedefimiz yaklaşık olarak karşıda görünü...,Kp5M0OHZqhc
1,burada da var burada da var burada da var bura...,24,31,ay geldim geldim geldim hoş geldiniz hoş geldi...,uPyoG7Y0Aas
2,misiniz içinde bir şeyler vardı ama içinde bir...,1420,1443,bugün hedefimiz yaklaşık olarak karşıda görünü...,Kp5M0OHZqhc
3,burada da var burada da var burada da var bura...,24,30,ay geldim geldim geldim hoş geldiniz hoş geldi...,uPyoG7Y0Aas
4,var burada da var burada da var burada var burası,506,517,bu persepolis e gidiyoruz buranın güneşe öyle ...,NeaWfoa3OgE
...,...,...,...,...,...
1074,o o o o o,97,99,yeni bir video ile daha yeniden merhaba değerl...,4zsu4gtRXbg
1075,o o o o o,791,794,bu iki saattir video başlayacağım ya bitir vid...,wXAnHYWPnfw
1076,o o o o o,785,788,bu videodaki altyazılar youtube tarafından oto...,FN3cAklpGqQ
1077,o o o o o,963,967,herkese merhaba kanalıma hoşgeldiniz bugün ve ...,WKcMQmdFU34


In [98]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,misiniz içinde bir şeyler vardı ama içinde bir...,1420,1445,bugün hedefimiz yaklaşık olarak karşıda görünü...,Kp5M0OHZqhc,https://www.youtube.com/watch?v=Kp5M0OHZqhc&t=...
1,burada da var burada da var burada da var bura...,24,31,ay geldim geldim geldim hoş geldiniz hoş geldi...,uPyoG7Y0Aas,https://www.youtube.com/watch?v=uPyoG7Y0Aas&t=24s
2,misiniz içinde bir şeyler vardı ama içinde bir...,1420,1443,bugün hedefimiz yaklaşık olarak karşıda görünü...,Kp5M0OHZqhc,https://www.youtube.com/watch?v=Kp5M0OHZqhc&t=...
3,burada da var burada da var burada da var bura...,24,30,ay geldim geldim geldim hoş geldiniz hoş geldi...,uPyoG7Y0Aas,https://www.youtube.com/watch?v=uPyoG7Y0Aas&t=24s
4,var burada da var burada da var burada var burası,506,517,bu persepolis e gidiyoruz buranın güneşe öyle ...,NeaWfoa3OgE,https://www.youtube.com/watch?v=NeaWfoa3OgE&t=...
...,...,...,...,...,...,...
1074,o o o o o,97,99,yeni bir video ile daha yeniden merhaba değerl...,4zsu4gtRXbg,https://www.youtube.com/watch?v=4zsu4gtRXbg&t=97s
1075,o o o o o,791,794,bu iki saattir video başlayacağım ya bitir vid...,wXAnHYWPnfw,https://www.youtube.com/watch?v=wXAnHYWPnfw&t=...
1076,o o o o o,785,788,bu videodaki altyazılar youtube tarafından oto...,FN3cAklpGqQ,https://www.youtube.com/watch?v=FN3cAklpGqQ&t=...
1077,o o o o o,963,967,herkese merhaba kanalıma hoşgeldiniz bugün ve ...,WKcMQmdFU34,https://www.youtube.com/watch?v=WKcMQmdFU34&t=...


In [99]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [100]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Word_Group_All_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [101]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

['Turkish_English_Word_Group_Adjust_In_Youtube_Sentence_Text_28_Word5.xlsx']

In [102]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [103]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass